## Naive Bayes on Political Text - Sarah Alqaysi

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [60]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

import os
import re
import emoji
import pandas as pd
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [61]:
convention_db = sqlite3.connect("/UserData/z003cu8m/OneDrive - Siemens AG/Documents/Documents/Documents/Siemens/Computer Backup/Personal/MSADS/Text Mining/Week four/2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [95]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text, party from conventions
                            ''')

#for row in query_results :
    # store the results in convention_data
 #   pass # remove this  

convention_data = list(query_results)
convention_data[:3]

[('Skip to content The Company Careers Press Freelancers Blog × Services Transcription Captions Foreign Subtitles Translation Freelancers About Contact Login « Return to Transcript Library home  Transcript Categories  All Transcripts 2020 Election Transcripts Classic Speech Transcripts Congressional Testimony & Hearing Transcripts Debate Transcripts Donald Trump Transcripts Entertainment Transcripts Financial Transcripts Interview Transcripts Political Transcripts Press Conference Transcripts Speech Transcripts Sports Transcripts Technology Transcripts Aug 21, 2020 2020 Democratic National Convention (DNC) Night 4 Transcript Rev  ›  Blog  ›  Transcripts  › 2020 Election Transcripts  ›  2020 Democratic National Convention (DNC) Night 4 Transcript Night 4 of the 2020 Democratic National Convention (DNC) on August 20. Read the full transcript of the event here. Transcribe Your Own Content  Try Rev for free  and save time transcribing, captioning, and subtitling.',
  'Democratic'),
 ('I’m 

Let's look at some random entries and see if they look right. 

In [94]:
random.choices(convention_data,k=10)[:3]

[('That was Prince Royce. Amazing. Shout out to the boogie down Bronx. In this next piece, you’ll hear about the vital role that women have played in moving us toward a more perfect union and breaking through that proverbial glass ceiling. And after that, you’ll hear from Secretary Hillary Rodham Clinton, the woman who put over 65 million cracks in it.',
  'Democratic'),
 ('[foreign language 00:24:07]. Relatives. As a First American and citizen of the Standing Rock Sioux tribe, I welcome you to the Paha Sapa, the Black Hills, the site of my creation story and home to the Oceti Sakowin, the great Sioux nation. We often say [foreign language 00:24:22]. We are all related. Our next president must lead by this philosophy for the betterment of our next seven generations. We cast 3 votes for Senator Bernie Sanders and 17 votes for our next President, Joe Biden.',
  'Democratic'),
 ('So what’s it going to be America? Alexandria Villaseñor: ( 50:58 )  Are you ready to vote for Joe Biden.',
  '

In [64]:
# converting the list into datafram for easier cleaning and tokenization

df_convention_data = pd.DataFrame(convention_data)
df_convention_data.columns = ['text', 'party']
df_convention_data.head(10)

,text,party
0,Skip to content The Company Careers Press Free...,Democratic
1,I’m here by calling the full session of the 48...,Democratic
2,"Every four years, we come together to reaffirm...",Democratic
3,We fight for a more perfect union because we a...,Democratic
4,"We must come together to defeat Donald Trump, ...",Democratic
5,Donald Trump is the wrong President for our co...,Democratic
6,My dad was a healthy 65 year old. His only pre...,Democratic
7,So it’s up to us to carry on the fight for jus...,Democratic
8,We just need leadership to bring us back toget...,Democratic
9,Next President right here.,Democratic


In [70]:
# Place any addtional functions or constants you need here. 

# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

# Stopwords
sw = stopwords.words("english")

# Two useful regex
whitespace_pattern = re.compile(r"\s+")
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")

# It's handy to have a full set of emojis
all_language_emojis = set()

for country in emoji.UNICODE_EMOJI : 
    for em in emoji.UNICODE_EMOJI[country] : 
        all_language_emojis.add(em)

# and now our functions
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity, and num_tokens most common
        tokens. Return a list of 
    """

    # Place your Module 2 solution here
    
    return(0)


    
def is_emoji(s):
    return(s in all_language_emojis)

def contains_emoji(s):
    
    s = str(s)
    emojis = [ch for ch in s if is_emoji(ch)]

    return(len(emojis) > 0)


def remove_stop(tokens) :
    # modify this function to remove stopwords
    return(tokens)
 
def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    # modify this function to return tokens
    return(text)

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)


In [71]:
# clean and tokenize the "text" data

my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]

df_convention_data["text"] = df_convention_data["text"].apply(prepare,pipeline=my_pipeline)
df_convention_data.head(10)

,text,party
0,skip to content the company careers press free...,Democratic
1,i’m here by calling the full session of the 48...,Democratic
2,every four years we come together to reaffirm ...,Democratic
3,we fight for a more perfect union because we a...,Democratic
4,we must come together to defeat donald trump a...,Democratic
5,donald trump is the wrong president for our co...,Democratic
6,my dad was a healthy 65 year old his only pree...,Democratic
7,so it’s up to us to carry on the fight for jus...,Democratic
8,we just need leadership to bring us back toget...,Democratic
9,next president right here,Democratic


In [96]:
# converting the dataframe back to a list

convention_data_list = df_convention_data.values.tolist()
convention_data_list[:3]

[['skip to content the company careers press freelancers blog × services transcription captions foreign subtitles translation freelancers about contact login « return to transcript library home  transcript categories  all transcripts 2020 election transcripts classic speech transcripts congressional testimony  hearing transcripts debate transcripts donald trump transcripts entertainment transcripts financial transcripts interview transcripts political transcripts press conference transcripts speech transcripts sports transcripts technology transcripts aug 21 2020 2020 democratic national convention dnc night 4 transcript rev  ›  blog  ›  transcripts  › 2020 election transcripts  ›  2020 democratic national convention dnc night 4 transcript night 4 of the 2020 democratic national convention dnc on august 20 read the full transcript of the event here transcribe your own content  try rev for free  and save time transcribing captioning and subtitling',
  'Democratic'],
 ['i’m here by calli

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [13]:
word_cutoff = 5

tokens = [w for t, p in convention_data_list for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2514 as features in the model.


In [76]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    
    ret_dict = dict()
    split = text.split()
    for feature_word in split: 
         if feature_word in fw:
            ret_dict[feature_word] = True 
             
    return(ret_dict)

In [ ]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [16]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data_list]

In [98]:
featuresets[:2]

[({'big': True,
   'thank': True,
   'you': True,
   'to': True,
   'everyone': True,
   'who': True,
   'brought': True,
   'school': True,
   'supplies': True,
   'our': True,
   'office': True,
   'last': True,
   'week': True,
   'executive': True,
   'director': True,
   'the': True,
   'other': True,
   'wonderful': True,
   'folks': True,
   'at': True,
   'communities': True,
   'in': True,
   'schools': True,
   'will': True,
   'make': True,
   'sure': True,
   'these': True,
   'are': True,
   'put': True,
   'use': True,
   'blessing': True,
   'lives': True,
   'of': True,
   'children': True},
  'Republican'),
 ({'with': True,
   'the': True,
   'incredible': True,
   'men': True,
   'and': True,
   'women': True,
   'of': True,
   'labor': True,
   'is': True,
   'one': True,
   'my': True,
   'life': True,
   'day': True,
   'was': True,
   'no': True,
   'we': True,
   'showed': True,
   'yet': True,
   'again': True,
   'who': True,
   'economy': True,
   'our': True,

In [27]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [28]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.444


In [64]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 liberal = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

_Your observations to come._



Answer: the most informative words are coming from republican tweets, which shows that they've utilized Twitter platform to support their campaigns. We can also see that Democrats are heavily focused on climate change, on the other hand, we see Republican addressing other topics like China, crime, religion, drugs... etc.

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [18]:
cong_db = sqlite3.connect("/UserData/z003cu8m/OneDrive - Siemens AG/Documents/Documents/Documents/Siemens/Computer Backup/Personal/MSADS/Text Mining/Week four/congressional_data.db")
cong_cur = cong_db.cursor()

In [19]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [20]:
# converting the list into dataframe for easier cleaning and tokenization

df_results = pd.DataFrame(results)
df_results.columns = ['candidate', 'party', 'tweet']
df_results.head(10)

,candidate,party,tweet
0,Mo Brooks,Republican,"b'""Brooks Joins Alabama Delegation in Voting A..."
1,Mo Brooks,Republican,"b'""Brooks: Senate Democrats Allowing President..."
2,Mo Brooks,Republican,"b'""NASA on the Square"" event this Sat. 11AM \x..."
3,Mo Brooks,Republican,"b'""The trouble with Socialism is that eventual..."
4,Mo Brooks,Republican,"b'""The trouble with socialism is eventually yo..."
5,Mo Brooks,Republican,"b'""U.S. Debt Biggest Threat to National Securi..."
6,Mo Brooks,Republican,b'# 1 jobs &amp; economic solution for America...
7,Mo Brooks,Republican,"b""#AL &amp; American hero Jeremiah Denton was ..."
8,Mo Brooks,Republican,"b""#AL05's loss is @POTUS's gain as my Legislat..."
9,Mo Brooks,Republican,b'#BorderCrisis: $3.7 billion problem or $20 m...


In [21]:
# clean and tokenizae the "tweets" in the dataframe

my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]

df_results["tweet"] = df_results["tweet"].apply(prepare,pipeline=my_pipeline)
df_results.head(10)

,candidate,party,tweet
0,Mo Brooks,Republican,bbrooks joins alabama delegation in voting aga...
1,Mo Brooks,Republican,bbrooks senate democrats allowing president to...
2,Mo Brooks,Republican,bnasa on the square event this sat 11am xe2x80...
3,Mo Brooks,Republican,bthe trouble with socialism is that eventually...
4,Mo Brooks,Republican,bthe trouble with socialism is eventually you ...
5,Mo Brooks,Republican,bus debt biggest threat to national security #...
6,Mo Brooks,Republican,b# 1 jobs amp economic solution for america is...
7,Mo Brooks,Republican,b#al amp american hero jeremiah denton was lai...
8,Mo Brooks,Republican,b#al05s loss is potuss gain as my legislative ...
9,Mo Brooks,Republican,b#bordercrisis 37 billion problem or 20 millio...


In [22]:
# removing the "candidate" column

df_results_new = df_results.drop(columns=['candidate'])
df_results_new.head()

,party,tweet
0,Republican,bbrooks joins alabama delegation in voting aga...
1,Republican,bbrooks senate democrats allowing president to...
2,Republican,bnasa on the square event this sat 11am xe2x80...
3,Republican,bthe trouble with socialism is that eventually...
4,Republican,bthe trouble with socialism is eventually you ...


In [23]:
# re-arranging the dataframe columns for our sublist

df_results_new = df_results_new.reindex(columns=['tweet','party'])
df_results_new.head()

,tweet,party
0,bbrooks joins alabama delegation in voting aga...,Republican
1,bbrooks senate democrats allowing president to...,Republican
2,bnasa on the square event this sat 11am xe2x80...,Republican
3,bthe trouble with socialism is that eventually...,Republican
4,bthe trouble with socialism is eventually you ...,Republican


In [58]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

tweet_data = df_results_new.values.tolist()

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [57]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [50]:
featuresets = [(conv_features(tweet,feature_words), party) for (tweet, party) in tweet_data]

In [51]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [52]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [56]:
for tweet, party in tweet_data_sample :    
    tweet = conv_features(tweet, feature_words) 
   
    # get the estimated party
    estimated_party = classifier.classify(tweet)
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: {'today': True, 'i': True, 'spoke': True, 'on': True, 'the': True, 'house': True, 'floor': True, 'protecting': True, 'health': True, 'care': True, 'for': True, 'women': True, 'and': True, 'their': True, 'work': True, 'coast': True}
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: {}
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: {'trump': True, 'thinks': True, 'its': True, 'just': True, 'too': True, 'easy': True, 'for': True, 'students': True, 'by': True, 'the': True, 'burden': True, 'of': True, 'debt': True, 'to': True, 'pay': True, 'off': True, 'student': True}
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: {'grateful': True, 'for': True, 'our': True, 'first': True, 'responders': True, 'rescue': True, 'police': True, 'and': True, 'who': True, 'have': True, 'been': True, 'working': True, 'tirelessly': True, 'to': True, 'keep': Tr

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [36]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

#for p in parties :
  #  for p1 in parties :
   #     results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    tweet = conv_features(tweet, feature_words) 
   
    # get the estimated party
    estimated_party = classifier.classify(tweet)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [37]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Democratic': 3801, 'Republican': 539}),
             'Democratic': defaultdict(int,
                         {'Democratic': 5529, 'Republican': 133})})

### Reflections

_Write a little about what you see in the results_ 

Answer: My model is heavily classifying tweets as Democratic than Repiblican, for both dictionaries.